In [33]:
import optuna
from keras.optimizers import Adam, RMSprop
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [34]:
import importlib
from __imports__ import *
import data, LSTM_returns, optimization
importlib.reload(data)

<module 'data' from 'c:\\Users\\ic2594\\crypto\\AAA_Thursday1\\data\\__init__.py'>

In [35]:
import numpy as np
import pandas as pd
import ta  # pip install ta

def compute_slope(series):
    y = series.values
    x = np.arange(len(y))
    if len(y) == 0:
        return np.nan
    return np.polyfit(x, y, 1)[0]

def add_technical_features(df, look_back=14):
    df = df.copy()  
    df['Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Risk'] = df['Return'].rolling(window=look_back).std()
    df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=look_back).rsi()
    df['ADX'] = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=look_back).adx()
    df['MOM'] = df['Close'] - df['Close'].shift(look_back)
    df['HL'] = df['High'] - df['Low']
    df['HO'] = df['High'] - df['Open']
    df['LO'] = df['Low'] - df['Open']
    df['buy_pressure_ratio'] = df['Taker buy quote asset volume'] / df['Quote asset volume']
    df['trades_per_volume'] = df['Number of trades'] / df['Quote asset volume']
    df['slope'] = df['Close'].rolling(window=look_back).apply(compute_slope, raw=False)
    df = df.dropna()
    cols_to_drop = [
        'Open', 'High', 'Low', 'Quote asset volume',
        'Number of trades', 'Taker buy base asset volume',
        'Taker buy quote asset volume', 'F&G'
    ]
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])
    return df

def prep_data(df, sequence_length=60, test_size=0.2, num_folds=4):
    # 1) include 'Close' since that's what we'll predict
    features = [
        'Close',      # ← our prediction target
        'Volume', 'F&G category', 'Return', 'Risk', 'RSI', 'ADX',
        'MOM', 'HL', 'HO', 'LO',
        'buy_pressure_ratio', 'trades_per_volume', 'slope'
    ]
    
    num_feats = [f for f in features if f != 'F&G category']
    df_num = df[num_feats]
    df_cat = pd.get_dummies(df['F&G category'], prefix='F_G')
    
    df_all = pd.concat([df_num, df_cat], axis=1).dropna()
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(df_all.values)
    feature_names = df_all.columns.tolist()
    
    target_idx = feature_names.index('Close')
    X, y = [], []
    for i in range(sequence_length, len(data_scaled)):
        X.append(data_scaled[i-sequence_length:i])
        # y is the scaled Close price at time i (next day)
        y.append(data_scaled[i, target_idx])
    X, y = np.array(X), np.array(y)
    
    # Split into folds
    fold_size = int(len(X) * (1 - test_size) / (num_folds + 1))
    folds = []
    for i in range(num_folds):
        start_idx = i * fold_size
        end_idx = start_idx + fold_size
        X_train = X[:end_idx]
        y_train = y[:end_idx]
        X_test = X[end_idx:end_idx + fold_size]
        y_test = y[end_idx:end_idx + fold_size]
        folds.append((X_train, X_test, y_train, y_test))
    
    # Final test set (last 20% of the data)
    final_test_idx = int(len(X) * (1 - test_size))
    X_final_test = X[final_test_idx:]
    y_final_test = y[final_test_idx:]
    
    return folds, (X_final_test, y_final_test), scaler


In [36]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import optuna

# ——— Hyperparameters (from Optuna) ———
UNITS_L0      = 160
DROPOUT_L0    = 0.0
LEARNING_RATE = 0.0007280355873484089
OPTIMIZER     = Adam(learning_rate=LEARNING_RATE)
BATCH_SIZE    = 32
MAX_EPOCHS    = 50

def train_lstm_for_asset(prepped_data, test_size=0.2):
    # … prep_data as before …
    (X_train, X_val, y_train, y_val), scaler = prepped_data
    
    # Build the model
    model = Sequential([
        Input(shape=X_train.shape[1:]),
        LSTM(UNITS_L0, return_sequences=False, dropout=DROPOUT_L0),
        Dense(1, activation='linear')
    ])
    
    # *** instantiate a new optimizer here, not reuse a global one ***
    optimizer = Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Train
    es = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=MAX_EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es],
        verbose=1
    )
    
    return model, scaler, history, (X_val, y_val)


def objective(trial, fold_number):
    # Suggest hyperparameters
    units_l0 = trial.suggest_int('units_l0', 50, 200)
    dropout_l0 = trial.suggest_float('dropout_l0', 0.0, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])
    max_epochs = trial.suggest_int('max_epochs', 10, 100)

    # Update global variables (if needed)
    global UNITS_L0, DROPOUT_L0, LEARNING_RATE, BATCH_SIZE, MAX_EPOCHS
    UNITS_L0 = units_l0
    DROPOUT_L0 = dropout_l0
    LEARNING_RATE = learning_rate
    BATCH_SIZE = batch_size
    MAX_EPOCHS = max_epochs

    # Train the model
    folds, (X_final_test, y_final_test), scaler = prep_data(df, sequence_length=30, test_size=0.2, num_folds=4)
    
    # Use walkforward folds for training and validation
    rmse_scores = []
    for fold_idx, (X_train, X_val, y_train, y_val) in enumerate(folds):
        model, scaler, history, (X_val, y_val) = train_lstm_for_asset(((X_train, X_val, y_train, y_val), scaler))
        preds = model.predict(X_val).flatten()
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        rmse_scores.append(rmse)

    print(f"MSE_SCORES: {rmse_scores}")
    
    # Return the average RMSE across folds
    return np.mean(rmse_scores)


# Example usage for any asset DataFrame `asset_df`:
# model, scaler, history, (X_val, y_val) = train_lstm_for_asset(asset_df)

# And to plot predictions vs actual:
# preds = model.predict(X_val).flatten()
# plt.figure(figsize=(10,5))
# plt.plot(y_val,  label='Actual Close (scaled)')
# plt.plot(preds, label='Predicted Close (scaled)')
# plt.legend()
# plt.title("LSTM Forecast (Scaled)")
# plt.show()

In [37]:
import os
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import load_model
class EarlyStoppingException(Exception):
    pass

# 1. Ensure eager mode for compatibility (if you set run_eagerly earlier)
tf.config.run_functions_eagerly(True)

# 2. Directories to save models and scalers
os.makedirs('models', exist_ok=True)
os.makedirs('scalers', exist_ok=True)

# 3. List of asset tickers
assets = ['SOLUSDT', 'BTCUSDT', 'ETHUSDT', 'DOGEUSDT', 'XRPUSDT']
results = {}

In [39]:
for ticker in assets:
    if ticker in results.keys():
        continue

    print(f"ENGINEERING {ticker}")

    # 4a) Load & feature-engineer
    df = data.load_asset(ticker, sampling='1d')
    df = data.add_fear_and_greed(df)
    df = add_technical_features(df)
    df = df[(df.index >= '2022-01-01') & (df.index <= '2023-12-31')]

    # 4b) Train the LSTM
    study = optuna.create_study(direction='minimize')
    
    def stopping_condition(study, trial):
        print(f"TICKER {ticker}")
        if study.best_value is not None and study.best_value < 0.045:
            print(f"Early stopping as the best score {study.best_value:.6f} is below 0.002")
            raise EarlyStoppingException()
            return True
        return False

    try:
        study.optimize(lambda trial: objective(trial, fold_number=4),
                    n_trials=100,
                    timeout=None,
                    callbacks=[stopping_condition])
    except EarlyStoppingException:
        print("Study stopped early due to custom stopping condition.")
    # Retrieve the best hyperparameters
    best_params = study.best_params
    print(f"Best parameters for {ticker}: {best_params}")

    # Update global hyperparameters with the best ones
    UNITS_L0 = best_params['units_l0']
    DROPOUT_L0 = best_params['dropout_l0']
    LEARNING_RATE = best_params['learning_rate']
    BATCH_SIZE = best_params['batch_size']
    MAX_EPOCHS = best_params['max_epochs']

    # Train the LSTM with the best hyperparameters
    folds, (X_final_test, y_final_test), scaler = prep_data(df, sequence_length=30, test_size=0.2, num_folds=4)
    (X_train, X_val, y_train, y_val) = folds[-1]
    X_train = np.concatenate((X_train, X_val), axis=0)
    y_train = np.concatenate((y_train, y_val), axis=0)
    model, scaler, history, (X_val, y_val) = train_lstm_for_asset(((X_train, X_final_test, y_train, y_final_test), scaler))

    # 4c) Save model and scaler
    model_path  = f'models/WF_{ticker}_lstm_model.h5'
    scaler_path = f'scalers/WF_{ticker}_scaler.pkl'
    model.save(model_path)
    joblib.dump(scaler, scaler_path)

    # 4d) Store paths and validation data
    results[ticker] = {
        'model_path':  model_path,
        'scaler_path': scaler_path,
        'history':     history,
        'X_val':       X_final_test,
        'y_val':       y_final_test
    }

# 5. Compute and print validation RMSE for each asset
for ticker, res in results.items():
    # Load model (if not in memory) and validation data
    model = load_model(res['model_path'])
    X_val = res['X_val']
    y_val = res['y_val']

    # Predict and score
    preds = model.predict(X_val).flatten()
    rmse  = np.sqrt(mean_squared_error(y_val, preds))
    print(f"{ticker} validation RMSE (scaled): {rmse:.4f}")


[I 2025-05-01 12:48:30,771] A new study created in memory with name: no-name-e897f123-5754-45f1-a004-1369620cedc4


ENGINEERING SOLUSDT
Epoch 1/38
1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.1731

C:\Users\ic2594\AppData\Local\Temp\ipykernel_14504\795467977.py:49: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step - loss: 0.1032 - val_loss: 0.1727
Epoch 2/38
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 0.0236 - val_loss: 0.0345
Epoch 3/38
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - loss: 0.0298 - val_loss: 0.0682
Epoch 4/38
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - loss: 0.0134 - val_loss: 0.1351
Epoch 5/38
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - loss: 0.0203 - val_loss: 0.0906
Epoch 6/38
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - loss: 0.0104 - val_loss: 0.0466
Epoch 7/38
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 0.0137 - val_loss: 0.0545
Epoch 8/38
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - loss: 0.0108 - val_loss: 0.0829
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Epoch 1/38
2/7 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.1094 

c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


...

MSE_SCORES: [np.float64(0.09875159041525683), np.float64(0.1280790776951652), np.float64(0.08372497307706393), np.float64(0.08640703655049144)]
TICKER SOLUSDT
Epoch 1/36
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 0.7304

C:\Users\ic2594\AppData\Local\Temp\ipykernel_14504\795467977.py:49: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - loss: 0.7103 - val_loss: 0.3420
Epoch 2/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - loss: 0.6123 - val_loss: 0.3298
Epoch 3/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - loss: 0.6262 - val_loss: 0.3178
Epoch 4/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 0.5721 - val_loss: 0.3062
Epoch 5/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 0.5390 - val_loss: 0.2949
Epoch 6/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - loss: 0.5436 - val_loss: 0.2840
Epoch 7/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - loss: 0.5548 - val_loss: 0.2733
Epoch 8/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step - loss: 0.5680 - val_loss: 0.2628
Epoch 9/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - loss: 0.5061 - val_loss: 0.2526
Epoch 10/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - loss: 0.5126 - val_loss: 0.2427
Epoch 11/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - loss: 0.4809 - val_loss: 0.2332
Epoch 12/36
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - loss: 0.4596 - val_loss: 0.2240


[W 2025-05-01 13:03:23,149] Trial 13 failed with parameters: {'units_l0': 52, 'dropout_l0': 0.22655432538037917, 'learning_rate': 3.929464630392763e-05, 'batch_size': 64, 'max_epochs': 36} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\ic2594\AppData\Local\Temp\ipykernel_14504\2040832433.py", line 25, in <lambda>
    study.optimize(lambda trial: objective(trial, fold_number=4),
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ic2594\AppData\Local\Temp\ipykernel_14504\795467977.py", line 67, in objective
    model, scaler, history, (X_val, y_val) = train_lstm_for_asset(((X_train, X_val, y_train, y_val), scaler))
                                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
test_predictions = {}

for ticker, res in results.items():
    model = load_model(res['model_path'])
    # Predict on the test set
    preds = model.predict(res['X_val']).flatten()
    dates = df.index[-len(preds):]  # Get the corresponding dates
    test_predictions[ticker] = pd.Series(preds, index=dates)

# Now `test_predictions` contains the predictions for each ticker indexed by date
%store test_predictions
joblib.dump(test_predictions, "models/test_predictions.pkl")

1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step

c:\Users\ic2594\AppData\Local\miniconda3\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
Stored 'test_predictions' (dict)


['models/test_predictions.pkl']